In [ ]:
import pandas as pd
import os
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define paths (Adjust 'Loan_Project' if you named your folder something else)
base_path = '/content/drive/MyDrive/Loan_Project'
raw_data_path = os.path.join(base_path, 'raw_data')

# 3. Verify files exist
print("Checking for files...")
if os.path.exists(raw_data_path):
    files = os.listdir(raw_data_path)
    print(f"Files found in {raw_data_path}:")
    for f in files:
        print(f" - {f}")
else:
    print(f"ERROR: Path {raw_data_path} not found. Check your folder name in Drive.")

# 4. Test Load (Load just 5 rows to check connection)
# We use error_bad_lines=False or on_bad_lines='skip' because these files sometimes have parsing errors
try:
    print("\nAttempting to load first 5 rows of Accepted Loans...")
    accepted_path = os.path.join(raw_data_path, 'accepted_2007_to_2018Q4.csv')
    df_test = pd.read_csv(accepted_path, nrows=5)
    print("Success! Columns detected:", list(df_test.columns[:5]), "...")
except Exception as e:
    print("Error loading data:", e)

Mounted at /content/drive
Checking for files...
Files found in /content/drive/MyDrive/Loan_Project/raw_data:
 - accepted_2007_to_2018Q4.csv
 - rejected_2007_to_2018Q4.csv

Attempting to load first 5 rows of Accepted Loans...
Success! Columns detected: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv'] ...


In [ ]:
import numpy as np

# --- CONFIGURATION ---
# To prevent RAM crashes, we will load only useful columns and a sample of rejected data
# We only need "Application" features (things known BEFORE the loan is given)

# Columns to keep from ACCEPTED data
accepted_cols = ['loan_amnt', 'title', 'dti', 'addr_state', 'emp_length', 'fico_range_low']

# Columns to keep from REJECTED data
rejected_cols = ['Amount Requested', 'Loan Title', 'Debt-To-Income Ratio', 'State', 'Employment Length', 'Risk_Score']

# --- 1. LOAD ACCEPTED LOANS ---
print("Loading Accepted Loans...")
# We use 'usecols' to save memory
df_acc = pd.read_csv(
    os.path.join(raw_data_path, 'accepted_2007_to_2018Q4.csv'),
    usecols=accepted_cols,
    dtype={'title': str, 'zip_code': str} # optimize types
)

# Rename to standard names
df_acc = df_acc.rename(columns={
    'fico_range_low': 'risk_score'
})

# Add Target Label: 1 = Approved
df_acc['is_approved'] = 1

print(f"Accepted loans loaded: {df_acc.shape}")

# --- 2. LOAD REJECTED APPLICATIONS ---
print("Loading Rejected Applications (Sampling)...")
# The rejected file is huge. Let's take a sample (e.g., 2 Million rows) to balance the data slightly
# or equal to the accepted size.
df_rej = pd.read_csv(
    os.path.join(raw_data_path, 'rejected_2007_to_2018Q4.csv'),
    usecols=rejected_cols,
    nrows=2000000  # Limit rows to save RAM. 2M is plenty for training.
)

# Rename to match the Accepted columns
df_rej = df_rej.rename(columns={
    'Amount Requested': 'loan_amnt',
    'Loan Title': 'title',
    'Debt-To-Income Ratio': 'dti',
    'State': 'addr_state',
    'Employment Length': 'emp_length',
    'Risk_Score': 'risk_score'
})

# Add Target Label: 0 = Rejected
df_rej['is_approved'] = 0

print(f"Rejected applications loaded: {df_rej.shape}")

# --- 3. MERGE AND CLEAN ---
print("Merging datasets...")
df_combined = pd.concat([df_acc, df_rej], axis=0, ignore_index=True)

# Free up memory by deleting old dataframes
del df_acc, df_rej
import gc
gc.collect()

print("Cleaning Data...")

# Clean DTI: Rejected data often has '20%', Accepted has 20.0
# Convert to string, strip %, convert to float
df_combined['dti'] = df_combined['dti'].astype(str).str.replace('%', '', regex=False)
df_combined['dti'] = pd.to_numeric(df_combined['dti'], errors='coerce')

# Clean Employment Length: Extract numbers (e.g., "10+ years" -> 10)
# We will just keep it simple categorical for now or map it later.
# For now, let's just handle missing values quickly
df_combined.dropna(subset=['dti', 'risk_score', 'loan_amnt'], inplace=True)

print(f"Final Combined Shape: {df_combined.shape}")
print(df_combined['is_approved'].value_counts())

# --- 4. SAVE TO DRIVE ---
# We save this as a processed file so we don't have to run this heavy step again
save_path = os.path.join(base_path, 'combined_eligibility_data.csv')
print(f"Saving to {save_path}...")
df_combined.to_csv(save_path, index=False)
print("Done! File saved.")

Loading Accepted Loans...
Accepted loans loaded: (2260701, 7)
Loading Rejected Applications (Sampling)...
Rejected applications loaded: (2000000, 7)
Merging datasets...
Cleaning Data...
Final Combined Shape: (3104227, 7)
is_approved
1    2258957
0     845270
Name: count, dtype: int64
Saving to /content/drive/MyDrive/Loan_Project/combined_eligibility_data.csv...
Done! File saved.


In [ ]:
df_combined.shape

(3104227, 7)

In [ ]:
df_combined.value_counts()

loan_amnt  emp_length  title        addr_state  dti    risk_score  is_approved
1000.0     < 1 year    other        CA          0.00   0.0         0              124
                                    FL          0.00   0.0         0              118
5000.0     < 1 year    other        CA          0.00   0.0         0              108
1000.0     < 1 year    other        TX          0.00   0.0         0              104
                                    NY          0.00   0.0         0              103
                                                                                 ... 
9500.0     < 1 year    credit_card  MO          10.74  729.0       0                1
                                                16.64  619.0       0                1
                                                22.88  756.0       0                1
                                                33.03  581.0       0                1
                                    MA          13.44  649.0       0                1
Name: count, Length: 2893405, dtype: int64

In [ ]:
# --- STEP 4: CREATE RISK (PD) DATASET ---

# 1. Define columns useful for Risk Modeling
# We need more financial details here than in the eligibility model
risk_cols = [
    'loan_status', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade',
    'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
    'verification_status', 'purpose', 'dti', 'earliest_cr_line',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies'
]

print("Loading Accepted Loans for Risk Modeling...")
df_risk = pd.read_csv(
    os.path.join(raw_data_path, 'accepted_2007_to_2018Q4.csv'),
    usecols=risk_cols,
    dtype={'term': str, 'grade': str, 'sub_grade': str}
)

print(f"Initial raw shape: {df_risk.shape}")

# 2. Filter for 'Completed' Loan Statuses only
# We cannot learn from 'Current' loans because the outcome is unknown
target_statuses = ['Fully Paid', 'Charged Off', 'Default']
df_risk = df_risk[df_risk['loan_status'].isin(target_statuses)].copy()

print(f"Shape after filtering for mature loans: {df_risk.shape}")

# 3. Create Binary Target
# 0 = Good (Fully Paid)
# 1 = Bad (Default / Charged Off)
# Note: In banking, '1' is usually the event we want to predict (Default)
df_risk['target_default'] = df_risk['loan_status'].apply(
    lambda x: 0 if x == 'Fully Paid' else 1
)

print("\nTarget Distribution (0=Paid, 1=Default):")
print(df_risk['target_default'].value_counts(normalize=True))

# 4. Save to Drive
risk_save_path = os.path.join(base_path, 'risk_data_clean.csv')
print(f"\nSaving risk data to {risk_save_path}...")
df_risk.to_csv(risk_save_path, index=False)
print("Done! Risk data saved.")


Loading Accepted Loans for Risk Modeling...
Initial raw shape: (2260701, 23)
Shape after filtering for mature loans: (1345350, 23)

Target Distribution (0=Paid, 1=Default):
target_default
0    0.80035
1    0.19965
Name: proportion, dtype: float64

Saving risk data to /content/drive/MyDrive/Loan_Project/risk_data_clean.csv...
Done! Risk data saved.
